In [1]:
#from ckiptagger import data_utils, construct_dictionary

# 1.載入需要的package，定義處理資料的function "word_proc_path"

In [1]:
import tensorflow as tf
import keras

import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import gc
import math

Using TensorFlow backend.


In [115]:
def word_proc_path(data_path,
              column_name,
              tt_seg = (1,10),
              export_flag = False, 
              #tt_begin = 0,tt_end = 50,
              output_file = "df_test_out.csv"):
    """
        data_path(str):資料路徑(字串)
        column_name(str):要處理的欄位名稱
        tt_seg(tuple):起訖列數
        export_flag(boolean):是否產出檔案
        output_file:輸出的檔案名稱，指定為CSV檔    
    """
    from ckiptagger import WS, POS, NER
    
    file_type = data_path[-4:]
    print("輸入檔案格式為 {}".format(file_type))
    if 'csv' in file_type:
        data = pd.read_csv(data_path)
    elif 'xls' in file_type:
        data = pd.read_excel(data_path)
    else:
        #print("檔案須為CSV或XLS/XLSX")
        raise Exception("檔案須為CSV或XLS/XLSX")
        #return None
    
    column_list = list(data.columns)
    if column_name in column_list:
        text = data[column_name]
    else:
        print("輸入欄位\"{input_column}\"不存在\n匯入檔案欄位{i_column_name}".format(input_column = column_name,
                                                                       i_column_name=str(column_list)))
        raise Exception("不存在此欄位名稱")

    
    #ls_Test = list(text[tt_begin:tt_end])
    print("起始列數：{begin}\n結束列數:{end}".format(begin = (tt_seg[0]-1),end = tt_seg[1]))
    ls_Test = list(text[(tt_seg[0]-1):tt_seg[1]])
    
    
    print("Got list\n 載入 WS 資料...")
    ws = WS("./data")
    print("============================\n 開始執行 WS\n============================\n")
    word_sentence_list = ws(ls_Test
                                # sentence_segmentation=True, # To consider delimiters
                                # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}),
                                # This is the defualt set of delimiters
                                # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
                                # coerce_dictionary = dictionary2, # words in this dictionary are forced
                            )
    del ws
    del WS
    gc.collect()
    print("============================\n WS 完成\n============================\n載入 POS 資料...")
    
    print("============================\n 開始執行 POS \n============================\n")
    pos = POS("./data")
    pos_sentence_list = pos(word_sentence_list)
    del pos
    del POS
    gc.collect()
    print("============================\n POS 完成\n============================\n載入NER 資料...")
    
    
    print("============================\n 開始執行 NER \n============================\n")
    ner = NER("./data")
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
    del NER
    del ner
    gc.collect()
    print("============================\n NER 完成\n============================\n")
        
    df_Test = pd.DataFrame(ls_Test,columns=[column_name])
    df_Test['WS'] = np.array(word_sentence_list)
    
    ##################
    #print("DEBUG")
    #print(pos_sentence_list[:5])
    ##################
    
    df_Test['POS'] = np.array(pos_sentence_list)
    df_Test['NER'] = np.array(entity_sentence_list)
    print("============================\n Dataframe Created \n============================\n")
    
    if export_flag is True:
        df_Test.to_csv(output_name)
        print("File Exported!!!!!")
    else:
        print("處理完成，不產生檔案")
        
    return df_Test,word_sentence_list,pos_sentence_list,entity_sentence_list

# 2.參數設定
### data_path_in    :要處理的檔案名稱(建議用絕對路徑)
### column_name_in  :要處理的欄位名稱
### proc_seg        :要處理的資料列起訖，格式為(起，迄)，均為整數，不可為0。
### export_flag     :是否要匯出檔案，True/False
### output_name     :匯出的檔案名稱，預設為CSV檔。

In [6]:
#data_path_in = "KCC_Data/KCCNews11554.csv"
data_path_in = "D:\python_kcc\\NER\剪報系統匯出-正確\\all-new(11554).xlsx"
#data_path_c = 'KCC_Data/20160419-20190804-自由(985筆).xlsx'
column_name_in = "Text"
column_name_in = '內容'
#column_name_c = "Teddd"
proc_seg = (1,500)
export_file = False
output_name = "mod_test.csv"

In [7]:
demo_df,ws_list,pos_list,ner_list = word_proc_path(data_path = data_path_in,
          column_name = column_name_in,
          tt_seg = proc_seg,export_flag = export_file ,output_file=output_name)

輸入檔案格式為 xlsx
起始列數：0
結束列數:500
Got list
 載入 WS 資料...
 開始執行 WS

 WS 完成
載入 POS 資料...
 開始執行 POS 

 POS 完成
載入NER 資料...
 開始執行 NER 

 NER 完成

 Dataframe Created 

處理完成，不產生檔案


In [8]:
len(ner_list),type(ner_list[0]),len(ner_list[9])

(500, set, 45)

# 3.取出NER類別及專有名詞，分別存到df_class_list 及df_ner_list中
# 4.將其組成有兩個欄位['類別','專有名詞']的DataFrame。

In [9]:
df_class_list = list()
df_ner_list = list()

count = 0
for i in range(len(ner_list)):
    for j in range(len(ner_list[i])):
        count = count +1
        a = ner_list[i].pop()
        df_class_list.append(a[2])
        df_ner_list.append(a[3])
        #print(type(new_list[j]),(new_list[j]))
print(count)

19013


In [10]:
extract_df = pd.DataFrame(
{'類別':df_class_list,
 '專有名詞':df_ner_list
})

# 5.檢視DataFrame資訊，查看各類別分別有多少資料

In [11]:
print(extract_df.describe())
extract_df.groupby('類別').count()

           類別   專有名詞
count   19013  19013
unique     18   6653
top       ORG     高雄
freq     4170    784


,專有名詞
類別,
CARDINAL,1894
DATE,2412
EVENT,302
FAC,806
GPE,3478
LANGUAGE,37
LAW,71
LOC,572
MONEY,363


# 6-1.DataFrame處理 ─ 刪除重複資料

In [12]:
dd_extract_df = extract_df.drop_duplicates()
len(extract_df),len(dd_extract_df)

(19013, 6875)

# 6-2.DataFrame處理 ─ 挑出需要匯出的類別

In [13]:
dd_extract_df['類別'].unique()

array(['FAC', 'ORDINAL', 'GPE', 'ORG', 'PERSON', 'MONEY', 'DATE',
       'PERCENT', 'LOC', 'TIME', 'NORP', 'CARDINAL', 'LANGUAGE', 'LAW',
       'EVENT', 'QUANTITY', 'WORK_OF_ART', 'PRODUCT'], dtype=object)

In [14]:
df_exp = dd_extract_df.loc[dd_extract_df['類別'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE'])]
len(df_exp)
#https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values

3984

# 6-3.DataFrame處理 ─ 刪掉長度小於2的專有名詞

In [15]:
df_exp_2 = df_exp[df_exp['專有名詞'].map(len) >= 2].copy()
len(df_exp_2)

3918

In [16]:
df_exp_2[:5].sort_values(by=['類別'])

,類別,專有名詞
2,GPE,台灣
3,ORG,文化部長
5,ORG,衛武營藝術文化中心
6,ORG,高雄大東文化藝術中心
7,PERSON,洪孟啟


# 7.將轉出的類別及專有名詞匯出成CSV檔
### (1)路徑：\\NER\\NER_export
### (2)檔案命名： NER_rlt_來源檔名_起始列數_結尾列數
### EX：來源檔名為KCCNews11554，本次處理的起訖列數為1~100
###     則檔名為 NER_rlt_KCCNews11554_1_100.csv


In [17]:
ex_NER_file_name =  "NER_rlt_all-new_1_500"
df_exp_2.sort_values(by=['類別']).to_csv("D:\python_kcc\\NER\\NER_export\\" + ex_NER_file_name + ".csv")

# 直接抓檔案來分段處理

In [4]:
full_path = r"D:\python_kcc\NER\剪報系統匯出-正確\all-new(11554).xlsx"

In [5]:
df_full_text = pd.read_excel(full_path)

In [7]:
df_full_text.columns

Index(['序號', '日期', '報別', '內容'], dtype='object')

In [6]:
df_seg_test = df_full_text['內容']

In [8]:
type(df_seg_test),df_seg_test[:5]

(pandas.core.series.Series,
 0    馬英九總統昨天上午視察衛武營藝術文化中心，文化部長洪孟啟、工程會副主委顏久榮及高雄市副市長吳...
 1    　日本九州熊本縣接連遭強震襲擊，台灣分別捐贈熊本縣政府和日本政府款項，高雄市長陳菊、台南市 ...
 2    　壽山動物園為增進動物飼養福祉及保育工作，特設立「高雄市壽山動物園認養辦法」設立動物認養專戶...
 3    日本九州熊本地震災情慘重，高雄市長陳菊昨天說，她將率先捐出一月所得；她同時指示市府相關單位 ...
 4    睽違超過半世紀，緬甸官方代表一行人，包含三位緬甸聯邦議會議員、美國農村發展組織及緬甸農業領袖...
 Name: 內容, dtype: object)

In [8]:
def proc_seg(in_df,seg_len=100):
    #計算輸入資料長度
    df_length = len(in_df)
    #計算要分成幾段處理
    seg_n = math.ceil(df_length/seg_len) 
    
    print("輸入資料長度:{in_len}\n共分成{seg}段".format(in_len = df_length,seg = seg_n))
    
    obj = {}
    for i in range(seg_n):
        #print(i*seg_len,(i+1)*seg_len)
        #print(df_seg_test[(i*100):(i+1)*100])
        obj['df_'+str(i)] = in_df[(i*seg_len):(i+1)*seg_len]   
        
   
    for segment in range(len(obj)):
        
        print("=======================\t開始處理第{}段\t===================\n".format(segment))
        print("第{}段長度：{}".format(('df_'+ str(segment)),len(obj['df_'+ str(segment)])))
        
        ########################################################################################
        # 呼叫word_proc_dataframe，回傳NER list
        ########################################################################################
        
        print("(segment*seg_len:{},(segment+1)*segment):{}".format((segment*seg_len),(segment+1)*seg_len))
        print("=====================================\b呼叫word_proc_dataframe\b=============================\n")
        entity_list = word_proc_dataframe(input_Series = obj['df_'+ str(segment)],
                            tt_seg = ((segment*seg_len),(segment+1)*seg_len),export_flag = False)
        
        print("=====================================呼叫 gen_df=============================\n")
        temp_df = gen_df(entity_list)
        temp_df.sort_values(by=['類別']).to_csv("D:\python_kcc\\NER\\NER_export\\df_out_" + str(segment) + ".csv")
        #print(type(obj['df_'+ str(segment)]))
        print("===========================\b第{}段處理完成\n================================================".format(segment))
        
        #print(obj['df_'+ str(segment)])
    

In [9]:
proc_seg(df_seg_test,400)

輸入資料長度:11554
共分成29段
=======================	開始處理第0段	===================

第df_0段長度：400
(segment*seg_len:0,(segment+1)*segment):400
====================================呼叫word_proc_datafram=============================

起始列數：0
結束列數:400
列印前5筆0    馬英九總統昨天上午視察衛武營藝術文化中心，文化部長洪孟啟、工程會副主委顏久榮及高雄市副市長吳...
1    　日本九州熊本縣接連遭強震襲擊，台灣分別捐贈熊本縣政府和日本政府款項，高雄市長陳菊、台南市 ...
2    　壽山動物園為增進動物飼養福祉及保育工作，特設立「高雄市壽山動物園認養辦法」設立動物認養專戶...
3    日本九州熊本地震災情慘重，高雄市長陳菊昨天說，她將率先捐出一月所得；她同時指示市府相關單位 ...
4    睽違超過半世紀，緬甸官方代表一行人，包含三位緬甸聯邦議會議員、美國農村發展組織及緬甸農業領袖...
Name: 內容, dtype: object
取得list，長度為：400
 載入 WS 資料...
 開始執行 WS

 WS 完成
載入 POS 資料...
 開始執行 POS 

 POS 完成
載入NER 資料...
 開始執行 NER 

 NER 完成

 Dataframe Created 

斷詞處理完成，不產生檔案
=====================================呼叫 gen_df=============================

==========================第0段處理完成
=======================	開始處理第1段	===================

第df_1段長度：400
(segment*seg_len:400,(segment+1)*segment):800
====================================呼叫word_proc_datafram=============================

起始列數：400

In [17]:
import glob

path = r'D:\python_kcc\NER\NER_export' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [19]:
print(len(frame))
frame.groupby('類別').count()

74162


,專有名詞
類別,
EVENT,2769
GPE,10630
LAW,1122
LOC,5124
ORG,28766
PERSON,25751


In [25]:
frame = frame.drop_duplicates().copy()
print(len(frame))
frame.groupby('類別').count()

37716


,專有名詞
類別,
EVENT,2009
GPE,3994
LAW,720
LOC,2586
ORG,14906
PERSON,13501


In [26]:
frame.sort_values(by=['類別']).to_csv("D:\python_kcc\\NER\\NER_export\\df_out_all.csv")